In [1]:
import pickle

with open('user_embedding_labels.pkl', 'rb') as lf:
    labels = pickle.load(lf)

with open('user_embedding_posts.pkl', 'rb') as pf:
    posts = pickle.load(pf)

In [2]:
# Tokenizing data
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

max_len = 1000
max_features = 10000
test_samples = 1500
max_words = 10000

Using TensorFlow backend.


In [3]:
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(posts)
sequences = tokenizer.texts_to_sequences(posts)
word_index = tokenizer.word_index
print('Found %s unique tokens' % len(word_index))

Found 116582 unique tokens


In [4]:
data = pad_sequences(sequences, maxlen=max_len)
labels = np.asarray(labels)

In [5]:
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (8675, 1000)
Shape of label tensor: (8675,)


In [6]:
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

In [7]:
type_dictionary = {
    '0000':'INTJ',
    '0001':'INTP',
    '0010':'INFJ',
    '0011':'INFP',
    '0100':'ISTJ',
    '0101':'ISTP',
    '0110':'ISFJ',
    '0111':'ISFP',
    '1000':'ENTJ',
    '1001':'ENTP',
    '1010':'ENFJ',
    '1011':'ENFP',
    '1100':'ESTJ',
    '1101':'ESTP',
    '1110':'ESFJ',
    '1111':'ESFP',
}

type_labels=['INTJ','INTP','INFJ','INFP','ISTJ','ISTP','ISFJ','ISFP', \
        'ENTJ','ENTP','ENFJ','ENFP','ESTJ','ESTP','ESFJ','ESFP',]

one_hot_types = []

for label in labels:
    bin_type = []
    
    if (label[0] == 'I'):
        bin_type.append(0)
    else:
        bin_type.append(1)
        
    if (label[1] == 'N'):
        bin_type.append(0)
    else:
        bin_type.append(1)
        
    if (label[2] == 'T'):
        bin_type.append(0)
    else:
        bin_type.append(1)
        
    if (label[3] == 'J'):
        bin_type.append(0)
    else:
        bin_type.append(1)
        
    one_hot_types.append(bin_type)

In [8]:
y = np.asarray(one_hot_types).astype('float32')

x_test = data[:test_samples]
y_test = y[:test_samples]

x_train = data[test_samples:]
y_train = y[test_samples:]

In [9]:
class_weights = {
    0:1.,
    1:3.43,
    2:1.,
    3:5.35,
    4:1.,
    5:1.12,
    6:1.,
    7:0.69
}

In [10]:
from keras.models import Sequential
from keras import layers, regularizers
from keras.optimizers import RMSprop

model = Sequential()
model.add(layers.Embedding(max_features, 128, input_length=max_len))
model.add(layers.Conv1D(32, 7, activation='relu', kernel_regularizer=regularizers.l2(0.001)))
model.add(layers.MaxPooling1D(5))
model.add(layers.Conv1D(32, 7, activation='relu', kernel_regularizer=regularizers.l2(0.001)))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(4))

model.compile(optimizer=RMSprop(lr=1e-4),
              loss='binary_crossentropy',
              metrics=['acc'])

history = model.fit(x_train, y_train,
                    epochs=30,
                    batch_size=128,
                    validation_split=0.2,
                    class_weight=class_weights)

Train on 5740 samples, validate on 1435 samples
Epoch 1/30
5740/5740 [==============================] - 33s 6ms/step - loss: 4.6159 - acc: 0.6222 - val_loss: 4.3367 - val_acc: 0.6218
Epoch 2/30
5740/5740 [==============================] - 31s 5ms/step - loss: 4.3198 - acc: 0.6402 - val_loss: 4.1765 - val_acc: 0.6768
Epoch 3/30
5740/5740 [==============================] - 32s 5ms/step - loss: 4.2329 - acc: 0.6714 - val_loss: 4.1497 - val_acc: 0.6768
Epoch 4/30
5740/5740 [==============================] - 32s 6ms/step - loss: 4.2148 - acc: 0.6714 - val_loss: 4.1485 - val_acc: 0.6768
Epoch 5/30
5740/5740 [==============================] - 32s 6ms/step - loss: 4.2019 - acc: 0.6714 - val_loss: 4.1468 - val_acc: 0.6768
Epoch 6/30
5740/5740 [==============================] - 31s 5ms/step - loss: 4.1905 - acc: 0.6714 - val_loss: 4.1492 - val_acc: 0.6768
Epoch 7/30
5740/5740 [==============================] - 32s 6ms/step - loss: 4.1807 - acc: 0.6714 - val_loss: 4.1458 - val_acc: 0.6768
Epoch 8

In [11]:
model = Sequential()
model.add(layers.Embedding(max_features, 128, input_length=max_len))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.MaxPooling1D(5))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(4))

model.compile(optimizer=RMSprop(lr=1e-4),
              loss='binary_crossentropy',
              metrics=['acc'])

model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=128,
                     class_weight=class_weights)

y_pred = model.predict(x_test)
results = model.evaluate(x_test, y_test)
results

Epoch 1/10
7175/7175 [==============================] - 38s 5ms/step - loss: 3.4756 - acc: 0.6221
Epoch 2/10
7175/7175 [==============================] - 38s 5ms/step - loss: 1.2704 - acc: 0.6221
Epoch 3/10
7175/7175 [==============================] - 37s 5ms/step - loss: 1.0304 - acc: 0.6709
Epoch 4/10
7175/7175 [==============================] - 39s 5ms/step - loss: 0.9932 - acc: 0.6725
Epoch 5/10
7175/7175 [==============================] - 37s 5ms/step - loss: 0.9774 - acc: 0.6725
Epoch 6/10
7175/7175 [==============================] - 37s 5ms/step - loss: 0.9630 - acc: 0.6725
Epoch 7/10
7175/7175 [==============================] - 37s 5ms/step - loss: 0.9490 - acc: 0.6725
Epoch 8/10
7175/7175 [==============================] - 37s 5ms/step - loss: 0.9352 - acc: 0.6725
Epoch 9/10
7175/7175 [==============================] - 37s 5ms/step - loss: 0.9214 - acc: 0.6725
Epoch 10/10
1500/1500 [==============================] - 3s 2ms/step


[0.6411504410107931, 0.6791666661898295]

In [12]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import itertools
import datetime

now = datetime.datetime.now()
pictures = []
folder = "1DCNN"

def byte_to_mbti(byte):
    binary = ''
    for letter in byte:
         binary +=(str(int(letter)))        
    return type_dictionary[binary]

decoded_y_true = [byte_to_mbti(label) for label in y_test.round()]
decoded_y_pred = [byte_to_mbti(label) for label in y_pred.round()]

matrix = confusion_matrix(decoded_y_true, decoded_y_pred, labels=type_labels)
matrix_I_E = confusion_matrix(y_test[:,0], y_pred[:,0].round(), labels=[0, 1])
matrix_N_S = confusion_matrix(y_test[:,1], y_pred[:,1].round(), labels=[0, 1])
matrix_T_F = confusion_matrix(y_test[:,2], y_pred[:,2].round(), labels=[0, 1])
matrix_J_P = confusion_matrix(y_test[:,3], y_pred[:,3].round(), labels=[0, 1])

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    
plt.figure(figsize=(15,10))
plot_confusion_matrix(matrix, classes=type_labels, normalize=True,
                      title='Confusion matrix')
name = str(folder) + "/" + str(now.month) + "-" + str(now.day) + "-" + \
str(now.hour) + "-" + str(now.minute) + "_" + "CM0.png"
plt.savefig(name)
plt.show()
pictures.append(name)

plt.clf()
plt.figure(figsize=(5,5))
plot_confusion_matrix(matrix_I_E, classes=["I", "E"], normalize=True,
                      title='I-E Confusion matrix')
name = str(folder) + "/" + str(now.month) + "-" + str(now.day) + "-" + \
str(now.hour) + "-" + str(now.minute) + "_" + "CM1.png"
plt.savefig(name)
plt.show()
pictures.append(name)

plt.clf()
plt.figure(figsize=(5,5))
plot_confusion_matrix(matrix_N_S, classes=["N", "S"], normalize=True,
                      title='N-S Confusion matrix')
name = str(folder) + "/" + str(now.month) + "-" + str(now.day) + "-" + \
str(now.hour) + "-" + str(now.minute) + "_" + "CM2.png"
plt.savefig(name)
plt.show()
pictures.append(name)

plt.clf()
plt.figure(figsize=(5,5))
plot_confusion_matrix(matrix_T_F, classes=["T", "F"], normalize=True,
                      title='T-F Confusion matrix')
name = str(folder) + "/" + str(now.month) + "-" + str(now.day) + "-" + \
str(now.hour) + "-" + str(now.minute) + "_" + "CM3.png"
plt.savefig(name)
plt.show()
pictures.append(name)

plt.clf()
plt.figure(figsize=(5,5))
plot_confusion_matrix(matrix_J_P, classes=["J", "P"], normalize=True,
                      title='J-P Confusion matrix')
name = str(folder) + "/" + str(now.month) + "-" + str(now.day) + "-" + \
str(now.hour) + "-" + str(now.minute) + "_" + "CM4.png"
plt.savefig(name)
plt.show()
pictures.append(name)

Normalized confusion matrix


<Figure size 1500x1000 with 2 Axes>

Normalized confusion matrix


<Figure size 640x480 with 0 Axes>

<Figure size 500x500 with 2 Axes>

...

<Figure size 640x480 with 0 Axes>

<Figure size 500x500 with 2 Axes>

Normalized confusion matrix


<Figure size 640x480 with 0 Axes>

<Figure size 500x500 with 2 Axes>